In [1]:
"""
Generating data for "Predicting Forest Regrowth in Amazonia given Land Use History"
Author: Ana Catarina Avila
Date: Dec 2023
"""

import ee
import geemap
#from geetools import batchvamo
# import pandas as pd
# import numpy as np
# from osgeo import gdal
# import os
# import datetime

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')


In [2]:
# Switches
region = "br_amazon"

# Choose year range
first_year = 1985
last_year = 2020 # year of biomass data

# how many meters around a secondary forest is considered a "neighborhood" for mature forest calculations
mat_neighbor = 1000

plot_region = ee.Geometry.Rectangle([-47, -3.5, -46, -2.5])
Map = geemap.Map()
Map.center_object(plot_region, zoom=9)

In [3]:
if region == "br_amazon":
    roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").filter(ee.Filter.eq("id", 18413)).geometry()
elif region == "br":
    roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_shapefile").geometry()
else:
    # land use for the pan amazonian dataset (spanning all amazonian countries)
    lulc = ee.Image("projects/mapbiomas-raisg/public/collection1/mapbiomas_raisg_panamazonia_collection1_integration_v1").byte()
    # note: there's less land use categories here than for the Brazilian territory.

# Load the images and feature collections
if not region == "panamaz":
  lulc = (ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1")
      .clip(roi)
      .select([f"classification_{year}" for year in range(first_year, last_year+1)])).byte()
  fire = (ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1")
      .clip(roi)
      .select([f"burned_coverage_{year}" for year in range(first_year, last_year)])).byte()

## Ages

Rewriting the code for "Benchmark maps of 33 years of secondary forest age for Brazil" to run with Collection 8 and include only class 3.

In [4]:
# making masks
# Anthropic, urban and Water Mask

LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9]
# INDEX ## 3 = forest
# 15 = pasture
# 39 = soy
# 20 = sugar cane
# 40 = rice
# 62 = cotton
# 41 = other temporary crop
# 46 = coffee
# 47 = citrus
# 35 = palm oil
# 48 = other perennial crop
# 9 = forest plantation
lulc_palette = ["#f1c232", "#FFFFB2", "#FFD966", "#E974ED", "#D5A6BD", "#e075ad", "#C27BA0", "#982c9e", "#e787f8", "#cd49e4", "#ad4413"]

empty = ee.Image().byte()

for i in range(first_year, last_year + 1):
    year = 'classification_' + str(i)
    anthropic = lulc.select(year).remap(LU_index, [1] * len(LU_index), 0).rename(year)
    empty = empty.addBands(anthropic)

anthropic_mask = empty.select(empty.bandNames().slice(1)) # we only want areas that DO show anthropic activity this year

# Replace 'YOUR_WATER_IMAGE_ID' with the actual water image ID you are working with
w_mask  = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("max_extent").clip(roi).remap([0,1],[1,0]);

urban = ee.Image("DLR/WSF/WSF2015/v1").clip(roi)
inverse_mask = urban.eq(1).Not()  # This will invert the mask, city pixels will be False (or 0) and non-urban pixels will be True (or 1)
urban = urban.updateMask(inverse_mask)  # This will replace 1 values (city pixels) with NA
urban_mask = urban.unmask(1)  # This will replace NA values (non-urban pixels) with 1

# Define a color palette
# palette = ['blue', 'red']  # Blue for 0, Red for 1
# vizpar2 = {'min': 1, 'max': len(LU_index), 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Add the layer to the map with the color palette
# Map.addLayer(w_mask, {'min': 0, 'max': 1, 'palette': palette}, 'w_mask')
# Map.addLayer(urban_mask, {'palette': palette}, 'urban_mask')
# Map.addLayer(anthropic_mask.select('classification_2020'), vizpar2, "anthropic")
# Map

In [5]:
# 1. Reclassifying MapBiomas Data # Step 1
empty = ee.Image().byte();

for i in range(first_year, last_year+1):
    year = 'classification_' + str(i)
    forest = lulc.select(year)
    forest = forest.remap([3,6], [1,1], 0) # Forest Formation and Flooded Forest classes from MapBiomas Project
    empty = empty.addBands(forest.rename(ee.String(year)))

mapbiomas_forest = empty.select(empty.bandNames().slice(1))

In [6]:
# 2. Mapping the Annual Increment of Secondary Forests # Step 2
regro = ee.Image().byte()
defor = ee.Image().byte()

for i in range(first_year, last_year):  # 1986-2020
    year1 = f'classification_{i}'
    year2 = f'classification_{i + 1}'
    a_mask = anthropic_mask.select(year1);
    forest1 = mapbiomas_forest.select(year1).remap([0, 1], [0, 2])  # Change forest pixels in 1985 to 2 years old
    forest2 = mapbiomas_forest.select(year2)
    # addition is 0 if was nonforest before and after; 1 if it was gained; 2 if it was forest before and then was lost; 3 if it was forest in both.
    sforest = forest1.add(forest2).multiply(a_mask).multiply(w_mask).multiply(urban_mask)
    for_gain = sforest.remap([0, 1, 2, 3], [0, 1, 0, 0]).rename(year2)
    for_loss = sforest.remap([0, 1, 2, 3], [0, 0, 1, 0]).rename(year2)
    regro = regro.addBands(for_gain)
    defor = defor.addBands(for_loss)

regro = regro.select(regro.bandNames().slice(1))  # Shows all years in which forest was gained.
# here, we could just mask by pixels that are forest in 2020 and find the year of last gain.

In [7]:
# 3. Mapping the Annual Extent of Secondary Forests # Step 3
extent = ee.Image().byte()
# add pixels that gained forest in 1986
extent = extent.addBands(regro.select('classification_1986').rename('classification_1986'))

for i in range(first_year + 1, last_year): #1987 to 2020
    year1 = f'classification_{i}' #1986
    year2 = f'classification_{i + 1}' #1987
    for_gain = regro.select(year2)
    acm_forest = extent.select(year1).add(for_gain) #pixels that gained forest in 1986 + pixels that gained forest in 1987
    old_values = list(range(37))
    new_values = [0, 1] + [1] * 35
    remap = acm_forest.remap(old_values, new_values)
    # mask (multiply) by pixels that were shown to be forest in 1987, hence eliminating any that may have regrown in 1986 but lost cover in 1987
    extent = extent.addBands(remap.multiply(mapbiomas_forest.select(year2)).rename(year2))

extent = extent.select(extent.bandNames().slice(1))

In [8]:
# 4. Calculating and Mapping the Age of Secondary Forests # Step 4
ages = ee.Image().byte()
ages = ages.addBands(extent.select('classification_1986').rename('classification_1986'))
ages = ages.slice(1) # remove "constant" band
age_total = ages # will use this as the "last total age" to keep iteratively adding values

for i in range(first_year + 1, last_year):
    year1 = f'classification_{i + 1}'# 1987-2020
    sforest = extent.select(year1) # forest cover in 1987
    age_total = age_total.add(sforest) # 1 year old forests in 1986 + cover in 1987
    f_year = mapbiomas_forest.select(year1)
    age_total = age_total.multiply(f_year) # mask by pixels that were forest that year, removing any forest loss
    ages = ages.addBands(age_total.rename(year1))

ages = ages.updateMask(ages) #keep only values as ages or NA

#ages range from 1 for those regrown in 2019-2020 to 35 for those regrown in 1985-1986
age = ages.select('classification_2020').rename('age')

# vizpar = {'min': 1, 'max': last_year - first_year, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(age, vizpar, "ages")
# Map

## Categorical Variables
Indigenous land, ecoregion, and soil type.

In [9]:
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))
ecoregions_img = ecoregions.reduceToImage(['ECO_ID'], ee.Reducer.first()).rename('ecoreg')

indig_land = ee.FeatureCollection("projects/ee-ana-zonia/assets/indig_land").filterBounds(roi)
indig_land_img = ee.Image().byte().paint(indig_land, 'gid').rename("indig").mask()

# palette = ['00FFFF', '0000FF']
# # Create a map
# Map = geemap.Map(center=[-10, -40], zoom=4)
# # Add the layer to the map with the color palette
# Map.addLayer(ecoregions_img, {'min': 0, 'max': 1000}, 'ecoregions_img')
# Map.addLayer(indig_land_img, {}, 'indig_land_img')
# # Display the map
# Map

Soil categories are strings, which Image objects can't handle. To transform the feature collection into an image, we convert the strings to integers by creating a dictionary.

In [10]:
soil = (ee.FeatureCollection('projects/ee-ana-zonia/assets/DSMW').filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

unique_domsoi = soil.aggregate_array('DOMSOI').distinct()

domsoi_dict = ee.Dictionary.fromLists(unique_domsoi, ee.List.sequence(1, unique_domsoi.length()))

soil = soil.remap(unique_domsoi, domsoi_dict.values(), 'DOMSOI')

soil_img = soil.reduceToImage(['DOMSOI'], ee.Reducer.first()).rename('soil')

# palette = ['00FFFF', '0000FF']
# # Create a map
# Map = geemap.Map(center=[-10, -40], zoom=4)
# # Add the layer to the map with the color palette
# Map.addLayer(soil_img, {'min': 0, 'max': 17, 'palette': palette}, 'soil_img')
# Map

## Biomass Edge Pixels
Get mean biomass for edge pixels

In [11]:
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass_2020").clip(roi)

# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age).float().rename('agbd')

# vizpar = {'min': 1, 'max': 415, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(biomass, vizpar, "biomass")
# Map

# img_export.append(biomass)

## Mature Forests

In [12]:
mature_mask = lulc.eq(3)
# Mask the image to keep only pixels with the value 3 in all bands
mature_mask = mature_mask.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask).rename('mat_biomass')

# # Compute the median biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer=ee.Reducer.median(), scale = 30, crs = mature_biomass.projection().crs())

# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['mat_biomass'], ee.Reducer.first())

# display('Median band values, Santa Cruz Mountains ecoregions', median_mature)
# Compute sum over round kernel
# assuming that 99.7% of seeds come from forests within mat_neighbor distance, that's 3 standard deviations
# sd = mat_neighbor/3
# kernel = ee.Kernel.gaussian(radius = mat_neighbor, sigma = sd, units='meters', normalize = True)
# weighed_sum_mature = mature_biomass.convolve(kernel).rename('weighted_sum_mature)

# vizpar = {'min': 0, 'max': 400, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(median_mature, vizpar, "biomass")
# Map

# img_export = img_export + [median_mature, weighed_sum_mature]

## Land use and land cover

In [13]:
#   last observed land use type - given secondary ages, get
# if age is 1 in 2020, it was not forest in 2019
# which means I want category in 2019
# if age is 35 in 2020, it was not forest in 1985
# which means I want category in 1985
lulc_masked = lulc.updateMask(age)

last_LU = ee.Image().constant(0).byte().rename('last_LU')

for i in range(first_year + 1, last_year):  # 1986-2020
  year = f'classification_{i}'
  age_mask = age.eq(last_year-i); #keep only the pixels with age equivalent to the correspondent year
  last_LU_observation = lulc.select(year).updateMask(age_mask) #keeps only land use classes of the year before abandonment
  last_LU = last_LU.add(last_LU_observation)

In [14]:
LU_sum = ee.Image().byte()

for val in LU_index:
  lulc_val = lulc_masked.eq(val)
  lulc_val_mask = lulc_val.mask()
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

LU_sum = LU_sum.slice(1).rename('LU_sum')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# vizpar_age = {'min': 1, 'max': last_year - first_year, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
#vizpar_lulc = {'min': 1, 'max': max(LU_index), 'palette': lulc_palette}
# Map.addLayer(age, vizpar_age, "ages")
# Map.addLayer(num_cells, vizpar_age, "lulc_masked")
# Map
#img_export = img_export + [last_LU, LU_sum]

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [15]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires')

# get fire frequency data from Mapbiomas - double check it.
# fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
# fire_freq = fire_freq.select('fire_frequency_1985_2020')

# how many years ago was each fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# # # Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(num_fires, {'min':0, 'max':35}, 'num_fires')
# Map

# img_export = img_export + [num_fires, time_since_all_fires, last_fire]

## Climate

### CWD

Calculate mean AET from MODIS and calculate CWD from the precipitation values as in Celso

Since MODIS only goes back to 2000, for now we are stuck with a fixed value for ET


In [19]:
# Function to calculate mean AET and add year property
Map = geemap.Map()
Map.center_object(plot_region, zoom=9)

# select for mature forests since the values can be put off by deforestation (causes lower ET)
# maybe it would be interesting to get mean ET per climatic region instead?

start = "2002-01-01"
end = "2020-12-01"

modis = (ee.ImageCollection("MODIS/061/MOD16A2GF")
         .filterDate(start, end)
         .select('ET', 'ET_QC').map(lambda image: image.clip(roi)))

# code sourced from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwise_extract(input, from_bit, to_bit):
   mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
   mask = ee.Number(1).leftShift(mask_size).subtract(1)
   return input.rightShift(from_bit).bitwiseAnd(mask)

def apply_QA_mask (image):
    QA = image.select('ET_QC')
    ET = image.select('ET').multiply(0.0125)  # multiply by the scale 0.1, divide by 8 to get daily info
    cloud_mask = bitwise_extract(QA, 3, 4).lte(0)
    qa_mask = bitwise_extract(QA, 5, 7).lte(1)
    mask = cloud_mask.And(qa_mask)
    return ET.updateMask(mask).set('system:time_start', image.get('system:time_start'))

# mask mature forests
modis_masked = modis.map(lambda image: image.mask(mature_mask))
# mask quality of pixels
modis_masked = modis_masked.map(apply_QA_mask)

# get mean value per month and ecoregion

months = range(1, 13)
monthly_et_list = []

# Loop through the months and filter images
for month in months:
    start_date = ee.Date.fromYMD(2002, month, 1)
    end_date = ee.Date.fromYMD(2020, month, 1)
    # print(end_date.getInfo())
    eight_day_images = modis_masked.select('ET').filter(ee.Filter.calendarRange(month, month, 'month')).filterDate(start_date, end_date)
    # print(eight_day_images.size().getInfo())
    monthly_et = eight_day_images.median().multiply(30)
    monthly_et_amaz = monthly_et.reduceRegion(roi, ee.Reducer.median(), scale = 500, bestEffort = True)
    monthly_et_list.append(monthly_et.set('month', month))






In [ ]:
# Function to calculate mean AET and add year property
Map = geemap.Map()
Map.center_object(plot_region, zoom=9)

vis_params = {
    'min': 0,
    'max': 150,
    'palette': ['006633', 'E5FFCC']
}

Map.addLayer(monthly_et_list[2], vis_params, 'Março')
Map.addLayer(monthly_et_list[5], vis_params, 'Junho')
Map.addLayer(monthly_et_list[8], vis_params, 'Sep')
Map.addLayer(monthly_et_list[11], vis_params, 'Dez')
Map


### Terraclim and seasonality

Bring temperature and precipitation and calculate seasonality

In [ ]:
terraclim = (ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
              .filterBounds(roi)
              .filterDate('1985-01-01', '2019-12-01'))

maxtemp = terraclim.select('tmmx').divide(10) # scale 0.1
mintemp = terraclim.select('tmmn').divide(10)
prec = terraclim.select('pr')


In [ ]:
water_deficit = prec.subtract(100)
print(terraclim.select(1).bandNames().getInfo())

months = range(1, 13)
years = range(1985, 2019)
monthly_wd_list = []

# Loop through the months and filter images
for year in years:
    for month in months:
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')
        monthly_rain = prec.filterDate(start_date, end_date)
        monthly_wd = monthly_rain.subtract(monthly_means_list[month])
        monthly_wd_list.append(monthly_wd.set('month', month))

# def calculate_cwd():
for i in range(1, length(monthly_wd_list)):
    wdn = monthly_wd_list[i]
    wdn1 = water_deficit[i-1] if i > 0 else 0
    if i == 0:
        water_deficit.select(i) = 0 if wdn > 0 else wdn
    else:
        cwd = wdn1 + wdn
        water_deficit.select(i) = cwd if cwd < 0 else 0


# Applying the Function
cwd = water_deficit.map()

# Determining the Annual MCDW
ano = 1985 # Start Year of the Temporal Series
for i in range(0, 132, 12): # Replace 132 by the Total Months of the Time Series
    cwd_a = cwd[i:(i+12)]
    mcwd_a = np.min(cwd_a)

In [ ]:

#### seasonality

# Calculate the mean monthly rainfall.
mean_monthly_rainfall = imageCollection.reduce(ee.Reducer.mean())

# Calculate the absolute deviations from the mean monthly rainfall.
absolute_deviations = imageCollection.map(lambda img: img.subtract(mean_monthly_rainfall).abs())

# Sum the absolute deviations.
sum_absolute_deviations = absolute_deviations.reduce(ee.Reducer.sum())

# Calculate the total annual precipitation.
total_annual_precipitation = imageCollection.reduce(ee.Reducer.sum())

# Calculate the SI index.
si_index = sum_absolute_deviations.divide(total_annual_precipitation)

# The result is an image where each pixel value is the SI index.
print(si_index.getInfo())



## Unify

In [ ]:
# cwd = ee.Image('projects/ee-ana-zonia/assets/cwd_chave').rename('cwd')
# proj = age.projection().getInfo()

#img_export.append(cwd)
# Add the 'ECO_ID' property of ecoregions as a new band in the 'age' image.
img_export = [age.pixelLonLat().float(), biomass, num_fires, last_fire, last_LU, ecoregions_img, indig_land_img]
# for eco_id in ecoregions.aggregate_array("ECO_ID").getInfo():
#     ecoreg =  ecoregions.filter(ee.Filter.eq("ECO_ID",eco_id))

## Export

In [80]:
img_export = age.addBands(biomass).selfMask().float()

fishnet = geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=0.5)
# Set 'id' as a property of the features
fishnet = fishnet.map(lambda feature: feature.set('id', feature.id()))
fishnet_info = fishnet.getInfo()['features']

for feature in fishnet_info:
    id = feature['id']
    grid_cell = fishnet.filter(ee.Filter.eq('id', id))
    img = img_export.clip(grid_cell)
    img = img.sample(grid_cell, scale = 30)
    task = ee.batch.Export.table.toDrive(
    collection=img,
    description=f'{id}_age_agbd',
    folder='drive_export',
    fileFormat='CSV')
    task.start()

# Map.addLayer(age_viz, {'min':0, 'max':35, 'palette':['blue', 'red']}, 'ages')
# vizpar = {'min': 1, 'max': 415, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Map.addLayer(agbd_viz, vizpar, "biomass")
# Map